In [1]:
import sys
import os
import pandas as pd
import cv2
import pickle
import EDA_utils as EU
import json

# query 실행
query1 = (f'''CREATE TABLE IF NOT EXISTS SysntheticHumanEyes_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(100) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(100) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        WhichEye VARCHAR(10) NOT NULL,
        
        ViewProjection VARCHAR(1000) NOT NULL,
        
        GazeWorld VARCHAR(500) NOT NULL,
        GazeImage VARCHAR(500) NOT NULL,
        
        EyeBlink VARCHAR(100) NOT NULL,
        EyeLookDown VARCHAR(100) NOT NULL,
        EyeLookIn VARCHAR(100) NOT NULL,
        EyeLookOut VARCHAR(100) NOT NULL,
        EyeLookUp VARCHAR(100) NOT NULL,
        EyeSquint VARCHAR(100) NOT NULL,
        EyeWide VARCHAR(100) NOT NULL,
        
        PupilOffset VARCHAR(500) NOT NULL,
        EyelidLower1 VARCHAR(500) NOT NULL,
        EyelidLower2 VARCHAR(500) NOT NULL,
        EyelidLower3 VARCHAR(500) NOT NULL,
        EyeCornerOuter1 VARCHAR(500) NOT NULL,
        EyeCornerOuter2 VARCHAR(500) NOT NULL,
        EyeCornerOuter3 VARCHAR(500) NOT NULL,
        EyeCornerInner1 VARCHAR(500) NOT NULL,
        EyeCornerInner2 VARCHAR(500) NOT NULL,
        EyeCornerInner3 VARCHAR(500) NOT NULL,
        Eye VARCHAR(500) NOT NULL,
        Pupil VARCHAR(500) NOT NULL,
        EyelidUpper1 VARCHAR(500) NOT NULL,
        EyelidUpper2 VARCHAR(500) NOT NULL,
        EyelidUpper3 VARCHAR(500) NOT NULL)''')
try:
    EU.MakeSQL.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

query2 = (f'''INSERT INTO SysntheticHumanEyes_info 
        (img_dir, img_format, img_width, img_height, color_space,
          label_dir, label_format, WhichEye,
          ViewProjection,
          GazeWorld, GazeImage,
          EyeBlink, EyeLookDown, EyeLookIn, EyeLookOut, EyeLookUp, EyeSquint, EyeWide,
          PupilOffset, EyelidLower1, EyelidLower2, EyelidLower3, EyeCornerOuter1, EyeCornerOuter2, EyeCornerOuter3, EyeCornerInner1, EyeCornerInner2, EyeCornerInner3,
          Eye, Pupil, EyelidUpper1, EyelidUpper2, EyelidUpper3) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''')

# label 경로
label_path = 'nsmount/Sysnthetic Human Eyes/data/RGB/labeldata'
image_path = 'nsmount/Sysnthetic Human Eyes/data/RGB/rawdata'

value_list1 = []
error_list = []

for label_name in os.listdir(label_path):
    label = os.path.join(label_path, label_name)
    try:
        with open(label, 'r') as f:
            texts = json.load(f)
            img_dir = label.replace('labeldata', 'rawdata').replace('.json', '.png')
            img_format = 'RGB'
            image = cv2.imread(img_dir)
            img_width = image.shape[1]
            img_height = image.shape[0]
            color_space = 'RGB'
            label_dir = label
            label_format = 'json'
            WhichEye = str(texts['Overall']['Side'])
            ViewProjection = str(texts['Camera']['ViewProjection'])
            GazeWorld = str(texts['Overall']['GazeWorld'])
            GazeImage = str(texts['Overall']['GazeImage'])
            EyeBlink = str(texts['Parameters']['EyeBlink'])
            EyeLookDown = str(texts['Parameters']['EyeLookDown'])
            EyeLookIn = str(texts['Parameters']['EyeLookIn'])
            EyeLookOut = str(texts['Parameters']['EyeLookOut'])
            EyeLookUp = str(texts['Parameters']['EyeLookUp'])
            EyeSquint = str(texts['Parameters']['EyeSquint'])
            EyeWide = str(texts['Parameters']['EyeWide'])
            PupilOffset = str(texts['Keypoints']['PupilOffset'])
            EyelidLower1 = str(texts['Keypoints']['EyelidLower1'])
            EyelidLower2 = str(texts['Keypoints']['EyelidLower2'])
            EyelidLower3 = str(texts['Keypoints']['EyelidLower3'])
            EyeCornerOuter1 = str(texts['Keypoints']['EyeCornerOuter'])
            EyeCornerOuter2 = str(texts['Keypoints']['EyeCornerOuter1'])
            EyeCornerOuter3 = str(texts['Keypoints']['EyeCornerOuter2'])
            EyeCornerInner1 = str(texts['Keypoints']['EyeCornerInner'])
            EyeCornerInner2 = str(texts['Keypoints']['EyeCornerInner1'])
            EyeCornerInner3 = str(texts['Keypoints']['EyeCornerInner2'])
            Eye = str(texts['Keypoints']['Eye'])
            Pupil = str(texts['Keypoints']['Pupil'])
            EyelidUpper1 = str(texts['Keypoints']['EyelidUpper1'])
            EyelidUpper2 = str(texts['Keypoints']['EyelidUpper2'])
            EyelidUpper3 = str(texts['Keypoints']['EyelidUpper3'])
            img_dir = img_dir.replace('nsmount', 'DataBase')
            label_dir = label_dir.replace('nsmount', 'DataBase')
            value = (img_dir, img_format, img_width, img_height, color_space, label_dir, label_format, WhichEye, ViewProjection, GazeWorld, GazeImage, EyeBlink, EyeLookDown, EyeLookIn, EyeLookOut, EyeLookUp, EyeSquint, EyeWide, PupilOffset, EyelidLower1, EyelidLower2, EyelidLower3, EyeCornerOuter1, EyeCornerOuter2, EyeCornerOuter3, EyeCornerInner1, EyeCornerInner2, EyeCornerInner3, Eye, Pupil, EyelidUpper1, EyelidUpper2, EyelidUpper3)
            value_list1.append(value)
    except Exception as ex:
        error_list.append((label, str(ex)))
        print(f"Error loading data from file: {label}")
        print(f"Error details: {ex}")

        # query 실행
    if len(value_list1) >= 300:
        print(value_list1[0])
        EU.MakeSQL.insert_dataset_values(query2, value_list1)
        value_list1 = []
        print(f'{len(value_list1)} / {len(label_name)}')

EU.MakeSQL.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')

create table done
('DataBase/Sysnthetic Human Eyes/data/RGB/rawdata/00000.png', 'RGB', 640, 360, 'RGB', 'DataBase/Sysnthetic Human Eyes/data/RGB/labeldata/00000.json', 'json', 'left', '[[-0.2183436006307602, -0.18288609385490417, 0, 0.9595844149589539], [0.9054615497589111, -0.3623201549053192, 0, 0.23742200434207916], [0.1666346937417984, 1.7290924787521362, 0, 0.15109123289585114], [69.3193359375, -328.9059753417969, 1, -98.38799285888672]]', '[-0.9493749737739563, -0.029563987627625465, -0.31275081634521484]', '[0.11043152607484653, 0.9938837346736188]', '0.82', '0.497', '0.071', '0', '0', '0', '0', '[98.95053100585938, -86.4355697631836, 181.93780517578125]', '[99.15290069580078, -87.179931640625, 181.69436645507812]', '[99.04150390625, -86.36966705322266, 181.56124877929688]', '[99.37469482421875, -85.55675506591797, 181.6800079345703]', '[99.93899536132812, -85.22795867919922, 181.71282958984375]', '[99.54624938964844, -85.17752838134766, 181.75869750976562]', '[99.91535949707031